In [121]:
from pennylane import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pennylane as qml
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [129]:
def powerFaultData(method = 'pca'):

	if method == 'pca':
		powerData = pd.read_csv('/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/Code/DR_data/pca3.csv')
		#powerData1 = powerData.where(powerData['fault'].isin([1])).dropna()[:10]
		#powerData2 = powerData.where(powerData['fault'].isin([6])).dropna()[:10]
		#powerData3 = powerData.where(powerData['fault'].isin([6])).dropna()[:10]
		#powerData4 = powerData.where(powerData['fault'].isin([6])).dropna()[:10]
		#powerData5 = powerData.where(powerData['fault'].isin([6])).dropna()[:10]
		#powerData6 = powerData.where(powerData['fault'].isin([6])).dropna()[:10]

		#powerData = pd.concat([powerData1,powerData2, powerData3, powerData4, powerData5, powerData6])
		
		X = []
		Y = []

		X = powerData[['pc1', 'pc2', 'pc3']]
		Y = powerData[['fault']]
		Y = np.asarray(Y).astype(int)

		xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.95, shuffle=True, random_state=42)
		xTrain = preprocessing.normalize(xTrain)
		xTest = preprocessing.normalize(xTest)

		return xTrain, yTrain
	
	if method == 'fda':
		powerData = pd.read_csv('/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/Code/DR_data/lda3.csv')
		
		X = []
		Y = []

		X = powerData[['LD1', 'LD2', 'LD3']]
		Y = powerData[['fault']]
		Y = np.asarray(Y).astype(int)

		xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.95, shuffle=True, random_state=42)
		xTrain = preprocessing.normalize(xTrain)
		xTest = preprocessing.normalize(xTest)

		return xTrain, yTrain

In [130]:
X, Y = powerFaultData('pca')
Y = Y.ravel()
print("Data: ", X[0])
print("Label: ", Y[0])

Data:  [-0.93400565  0.33191297  0.13216368]
Label:  5


In [131]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [132]:
dev = qml.device("default.qubit", wires=3, shots=None)
wires = dev.wires.tolist()

def layer(x, params, wires, i0=0, inc=1):
    
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)] * params[2, 0], wires=[wire])
        #qml.RZ(x[i], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])


def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))
        

adjoint_ansatz = qml.adjoint(ansatz)

def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    params = np.random.uniform(0, 2 * np.pi, (1, num_layers * num_wires), requires_grad=True)
    return np.random.uniform(0, 2 * np.pi, (num_layers, 3, num_wires), requires_grad=True)

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [133]:
init_params = random_params(num_wires= 3, num_layers = 6)
len(init_params[0])

3

In [134]:
kernel_value = kernel(X[0], X[1], init_params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")
print('---------------------------------------------------------------------------------------------')
print('Quantum Circuit: ')
drawer = qml.draw(kernel_circuit)
print(drawer(X[0], X[1], init_params))
print('---------------------------------------------------------------------------------------------')

The kernel value between the first and second datapoint is 0.074
---------------------------------------------------------------------------------------------
Quantum Circuit: 
0: ──H──RZ(-5.09)──RY(1.38)─╭●──────────────────╭RZ(1.61)──H──RZ(-4.58)──RY(1.88)─╭●───────
1: ──H──RZ(1.81)───RY(0.61)─╰RZ(4.30)─╭●────────│──────────H──RZ(1.63)───RY(4.43)─╰RZ(4.43)
2: ──H──RZ(0.72)───RY(1.43)───────────╰RZ(5.79)─╰●─────────H──RZ(0.65)───RY(4.80)──────────

────────────╭RZ(4.31)──H──RZ(-3.79)──RY(1.42)─╭●──────────────────╭RZ(1.30)──H──RZ(-3.32)──RY(0.98)
──╭●────────│──────────H──RZ(1.35)───RY(5.12)─╰RZ(3.94)─╭●────────│──────────H──RZ(1.18)───RY(0.86)
──╰RZ(0.09)─╰●─────────H──RZ(0.54)───RY(5.76)───────────╰RZ(1.91)─╰●─────────H──RZ(0.47)───RY(1.51)

──╭●──────────────────╭RZ(2.04)──H──RZ(-4.62)──RY(4.54)─╭●──────────────────╭RZ(5.66)──H──RZ(-1.47)
──╰RZ(5.62)─╭●────────│──────────H──RZ(1.64)───RY(1.86)─╰RZ(5.84)─╭●────────│──────────H──RZ(0.52)─
────────────╰RZ(2.38)─╰●─────────H──RZ(0.65)─

In [135]:
trained_kernel = lambda x1, x2: kernel(x1, x2, init_params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

In [119]:
print("Kernel Alignment with Gradient Descent")
params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    subset = np.random.choice(list(range(len(X))), 4)
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

Kernel Alignment with Gradient Descent
Step 10 - Alignment = 0.822
Step 20 - Alignment = 0.855
Step 30 - Alignment = 0.875
Step 40 - Alignment = 0.879
Step 50 - Alignment = 0.892
Step 60 - Alignment = 0.900
Step 70 - Alignment = 0.908
Step 80 - Alignment = 0.920
Step 90 - Alignment = 0.921
Step 100 - Alignment = 0.928
Step 110 - Alignment = 0.939
Step 120 - Alignment = 0.942
Step 130 - Alignment = 0.946
Step 140 - Alignment = 0.944
Step 150 - Alignment = 0.948
Step 160 - Alignment = 0.946
Step 170 - Alignment = 0.950
Step 180 - Alignment = 0.953
Step 190 - Alignment = 0.953
Step 200 - Alignment = 0.956
Step 210 - Alignment = 0.959
Step 220 - Alignment = 0.957
Step 230 - Alignment = 0.957
Step 240 - Alignment = 0.957
Step 250 - Alignment = 0.960
Step 260 - Alignment = 0.959
Step 270 - Alignment = 0.957
Step 280 - Alignment = 0.959
Step 290 - Alignment = 0.959
Step 300 - Alignment = 0.963
Step 310 - Alignment = 0.965
Step 320 - Alignment = 0.965
Step 330 - Alignment = 0.967
Step 340 - Al

In [ ]:
def uncertinitySamplingSubset(X, svm_trained, subSize, sampling = 'entropy'):
	
	if sampling == 'entropy':
		probabilities = svm_trained.predict_proba(X)
		entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
		selected_indices = np.argsort(entropy)[:subSize]

		return selected_indices
	
	return None

In [ ]:
print("Kernel Alignment with Gradient Descent with Active Learning")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix, probability=True)
svm_trained.fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    
    #subset = np.random.choice(list(range(len(X))), 4) 
    trained_kernel = lambda x1, x2: kernel(x1, x2, params)
    trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
    svm_trained = SVC(kernel=trained_kernel_matrix, probability=True).fit(X, Y)
    
    subset = uncertinitySamplingSubset(X, svm_trained=svm_trained, subSize=4)

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")